In [ ]:
import numpy as np
import random
import time
import math
import random
import os
import json
import sys
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt

# Notebook path
current_directory = os.getcwd()
# Get one behind for calling the other modules
root_directory = os.path.abspath(os.path.join(current_directory, '..'))
sys.path.append(root_directory)

#Network and measures
from to_get_data.net_construction_vis_measures import *

from to_get_data.generate_database import *
#Agrobiodiversity
from to_get_data.agrobiodiversity_data import *

from model_class.Variety import Variety
from model_class.UD import *
from model_class.UD_aleatory_inheritance import *

# Dual organization

In [ ]:
#DUAL ORGANIZATION
#Parameters dual_organization
experiments = 2
iniciais = 30 # Initial UD
prob_morte = 0.06 # Beta from the gutertz equation
mediao = 4 # Average children per UD
random.seed(42) # Random seed
t = 501 #steps of the simulation
view = 50
inheritance = [0,25,50,75,100, 'False']
data_inherit_dual = {}
data_step_mean_var_ud = {}
data_step_var_system = {}
data_step_unique_system = {}
for inherit in inheritance:
    data_step_mean_var_ud[inherit] = []
    data_step_var_system[inherit] = []
    data_step_unique_system[inherit] = []
    media = mediao
    print(f"Experiments with inheritance {inherit}")
    data_dual_mean_ud = []
    data_dual_com_level = []
    data_dual_total_varieties = []
    original_media = media
    for exp in range(1,experiments+1):
        mean_var_ud = []
        var_system = []
        unique_system = []
        #print(media)
        #### Initialization dual_organization
        ## Communities
        clans = (list(range(1, 2 + 1)) * (iniciais // 2)) # Just 2 clans
        communities = (list(range(1, 2 + 1)) * (iniciais // 2)) # Just 2 communities
        random.shuffle(clans)
        random.shuffle(communities)
        if inherit != 'False':
            UD_dual_organization.uds = {}
            instancias_ud = []
            for id_ud in range(1, int(iniciais) + 1):
                    instancias_ud.append(UD_dual_organization(id_ud, media, communities.pop(), clans.pop()))
            ## Varieties
            limited_varieties = True
            if limited_varieties == False:
                for id_ud, ud in UD_dual_organization.uds.items():
                    for _ in range(3):
                        ud.varieties.append(Variety())
            else:
                initial_varieties = [Variety() for _ in range (10)]
                for ud in UD_dual_organization.uds.values():
                    random_samples = random.sample(initial_varieties, 3)
                    for sample in random_samples:
                        ud.varieties.append(sample)
            #### Simulation
            for i in range (1, (t)):
                uds_copy = {id_ud: ud for id_ud, ud in UD_dual_organization.uds.items() if ud.activa}
                alive = len(uds_copy)
                if alive > 200 and (media == original_media): #or media == original_media-1):
                    media -= media//2
                    if media >= 4:
                        media = media //2
                    #print(f' no ano {i} temos a media {media}')
                elif alive < 150 and media < original_media:
                    media = original_media
                    #print(f' no ano {i} temos a media {media}')
                if not uds_copy:
                    print("acabo")
                    break
                # CYcle
                for id_ud, ud in uds_copy.items():
                    ud.ter_filho()
                    ud.buscar_ud(media, inherit, True, False)
                    ud.incrementar_idade()
                    ud.death_probability(prob_morte)
                if i%view == 0:
                    var_ud = []
                    var_total = []
                    for ud in [ud for ud in UD_endogamy.uds.values() if ud.activa ==True]:
                        var_ud.append(len(ud.varieties))
                        for variety in ud.varieties:
                            var_total.append(variety.variety_id)
                    mean_var_ud.append(np.mean(var_ud))
                    var_system.append(len(var_total))
                    unique_system.append(len(set(var_total))) 
            #Get the distribution, community diversity and mean by UD
            total_varieties = []
            varieties_per_UD = []
            for ud in [ud for ud in UD_dual_organization.uds.values() if ud.activa ==True]:
                varieties_per_UD.append(len(ud.varieties))
                for variety in ud.varieties:
                    total_varieties.append(variety.variety_id)
            #mean_ud = np.mean(varieties_per_UD)
            #diversity_comm_level = len(set(total_varieties))
            data_dual_mean_ud.append(np.mean(varieties_per_UD))
            data_dual_com_level.append(len(set(total_varieties)))
            data_dual_total_varieties.append(total_varieties)
        #data_inherit_dual[inherit] = [data_dual_mean_ud, data_dual_com_level, data_dual_total_varieties]
        else:
            UD_dual_organization_al_inh.uds = {}
            instancias_ud = []
            for id_ud in range(1, int(iniciais) + 1):
                    instancias_ud.append(UD_dual_organization_al_inh(id_ud, media, communities.pop(), clans.pop()))
            ## Varieties
            limited_varieties = True
            if limited_varieties == False:
                for id_ud, ud in UD_dual_organization_al_inh.uds.items():
                    for _ in range(3):
                        ud.varieties.append(Variety())
            else:
                initial_varieties = [Variety() for _ in range (10)]
                for ud in UD_dual_organization_al_inh.uds.values():
                    random_samples = random.sample(initial_varieties, 3)
                    for sample in random_samples:
                        ud.varieties.append(sample)
            #### Simulation
            for i in range (1, (t)):
                uds_copy = {id_ud: ud for id_ud, ud in UD_dual_organization_al_inh.uds.items() if ud.activa}
                alive = len(uds_copy)
                if alive > 200 and (media == original_media): #or media == original_media-1):
                    media -= media//2
                    if media >= 4:
                        media = media //2
                    #print(media)
                elif alive < 150 and media < original_media:
                    media = original_media
                    #print(media)
                if not uds_copy:
                    print("acabo")
                    break
                # Cycle: 
                varieties_list = [var for ud in uds_copy.values() for var in ud.varieties]
                for id_ud, ud in uds_copy.items():
                    ud.ter_filho()
                    ud.buscar_ud(media, varieties_list, True, False)
                    ud.incrementar_idade()
                    ud.death_probability(prob_morte)
                if i%view == 0:
                    var_ud = []
                    var_total = []
                    for ud in [ud for ud in UD_endogamy.uds.values() if ud.activa ==True]:
                        var_ud.append(len(ud.varieties))
                        for variety in ud.varieties:
                            var_total.append(variety.variety_id)
                    mean_var_ud.append(np.mean(var_ud))
                    var_system.append(len(var_total))
                    unique_system.append(len(set(var_total))) 
            #Get the distribution, community diversity and mean by UD
            #print(len([ud for id, ud in uds_copy.items() if ud.activa]))
            #Get the distribution, community diversity and mean by UD
            total_varieties = []
            varieties_per_UD = []
            for ud in [ud for ud in UD_dual_organization_al_inh.uds.values() if ud.activa ==True]:
                varieties_per_UD.append(len(ud.varieties))
                for variety in ud.varieties:
                    total_varieties.append(variety.variety_id)
            #mean_ud = np.mean(varieties_per_UD)
            #diversity_comm_level = len(set(total_varieties))
            data_dual_mean_ud.append(np.mean(varieties_per_UD))
            data_dual_com_level.append(len(set(total_varieties)))
            data_dual_total_varieties.append(total_varieties)
        data_step_var_system[inherit].append(var_system)
        data_step_unique_system[inherit].append(unique_system)
        data_step_mean_var_ud[inherit].append(mean_var_ud)
        data_inherit_dual[inherit] = [data_dual_mean_ud, data_dual_com_level, data_dual_total_varieties]
#CUIDADO, NO SOBREESCRIBIR
        # # Folder
# folder_name = 'results_inheritance_aleatory'
# if not os.path.exists(folder_name):
#     os.makedirs(folder_name)
# # Inherit data
# inherit_dual_path = os.path.join(folder_name, 'inherit_data_dual.json')
# # Save
# with open(inherit_dual_path, 'w') as json_file:
#     json.dump(data_inherit_dual, json_file, indent=4)
# # Confirmation message
# print("Data saved successfully to", inherit_dual_path)

In [ ]:
## Data and plot from loaded data
mean_div_com = []
std_div_com = []
inheritance = ['0','25','50','75','100', 'False']
#inheritance = [100,90,80,70,60,50, False]
for i in inheritance:
    mean_div_com.append(np.mean(loaded_data_inherit_dual[i][1]))
    std_div_com.append(np.std(loaded_data_inherit_dual[i][1]))
#Plot
inheritance_possibilities = ['0','25','50','75','100', 'Aleatory']
plt.errorbar(inheritance_possibilities, mean_div_com, yerr=std_div_com, fmt='o', capsize=5, capthick=2, ecolor='red', label='Data with Error Bars')
plt.xlabel('Percentage of matrilineal inheritance')
plt.ylabel('Mean')
plt.ylim(bottom=0)
plt.title('Diversity at the community level with dual organization')
plt.legend()
plt.grid(True)
plt.show()

# Restricted

In [ ]:
#UD_restricted
#Parameters restricted_exchange
experiments = 2
iniciais = 30 # Initial UD
prob_morte = 0.06 # Beta from the gutertz equation
mediao = 4 # Average children per UD
#random.seed(42) # Random seed
t = 501 #steps of the simulation
inheritance = [0,25,50,75,100, 'False']
data_inherit_restricted = {}
for inherit in inheritance:
    media = mediao
    print(f"Experiments with inheritance {inherit}")
    data_restricted_mean_ud = []
    data_restricted_com_level = []
    data_restricted_total_varieties = []
    original_media = media
    for exp in range (1, experiments+1):
        #### Initialization restricted_exchange
        ## Communities
        clans = (list(range(1, 3 + 1)) * (iniciais // 3)) # Just 3 clans
        communities = (list(range(1, 2 + 1)) * (iniciais // 2)) # Just 2 communities
        random.shuffle(clans)
        random.shuffle(communities)
        if inherit != 'False':
            UD_restricted.uds = {}
            instancias_ud = []
            for id_ud in range(1, int(iniciais) + 1):
                    instancias_ud.append(UD_restricted(id_ud, media, communities.pop(), clans.pop()))
            ## Varieties
            limited_varieties = True
            if limited_varieties == False:
                for id_ud, ud in UD_restricted.uds.items():
                    for _ in range(3):
                        ud.varieties.append(Variety())
            else:
                initial_varieties = [Variety() for _ in range (10)]
                for ud in UD_restricted.uds.values():
                    random_samples = random.sample(initial_varieties, 3)
                    for sample in random_samples:
                        ud.varieties.append(sample)
            #### Simulation
            for i in range (1, (t)):
                uds_copy = {id_ud: ud for id_ud, ud in UD_restricted.uds.items() if ud.activa}
                alive = len(uds_copy)
                if alive > 200 and (media == original_media): #or media == original_media-1):
                    media -= media//2
                    if media >= 4:
                        media = media //2
                    #print(media)
                elif alive < 150 and media < original_media:
                    media = original_media
                    #print(media)
                if not uds_copy:
                    print("acabo")
                    break
                for id_ud, ud in uds_copy.items():
                    ud.ter_filho()
                    ud.buscar_ud(media, inherit, True, False)
                    ud.incrementar_idade()
                    ud.death_probability(prob_morte)
            #Get the distribution, community diversity and mean by UD
            #print(len([ud for id, ud in uds_copy.items() if ud.activa]))
            #Get the distribution, community diversity and mean by UD
            total_varieties = []
            varieties_per_UD = []
            for ud in [ud for ud in UD_restricted.uds.values() if ud.activa ==True]:
                varieties_per_UD.append(len(ud.varieties))
                for variety in ud.varieties:
                    total_varieties.append(variety.variety_id)
            #mean_ud = np.mean(varieties_per_UD)
            #diversity_comm_level = len(set(total_varieties))
            data_restricted_mean_ud.append(np.mean(varieties_per_UD))
            data_restricted_com_level.append(len(set(total_varieties)))
            data_restricted_total_varieties.append(total_varieties)
        #data_inherit_restricted[inherit] = [data_restricted_mean_ud, data_restricted_com_level, data_restricted_total_varieties]
        else:
            UD_restricted_al_inh.uds = {}
            instancias_ud = []
            for id_ud in range(1, int(iniciais) + 1):
                    instancias_ud.append(UD_restricted_al_inh(id_ud, media, communities.pop(), clans.pop()))
            ## Varieties
            limited_varieties = True
            if limited_varieties == False:
                for id_ud, ud in UD_restricted_al_inh.uds.items():
                    for _ in range(3):
                        ud.varieties.append(Variety())
            else:
                initial_varieties = [Variety() for _ in range (10)]
                for ud in UD_restricted_al_inh.uds.values():
                    random_samples = random.sample(initial_varieties, 3)
                    for sample in random_samples:
                        ud.varieties.append(sample)
            #### Simulation
            for i in range (1, (t)):
                uds_copy = {id_ud: ud for id_ud, ud in UD_restricted_al_inh.uds.items() if ud.activa}
                alive = len(uds_copy)
                if alive > 200 and (media == original_media): #or media == original_media-1):
                    media -= media//2
                    if media >= 4:
                        media = media //2
                    #print(media)
                elif alive < 150 and media < original_media:
                    media = original_media
                    #print(media)
                if not uds_copy:
                    print("acabo")
                    break
                # Cycle: 
                varieties_list = [var for ud in uds_copy.values() for var in ud.varieties]
                for id_ud, ud in uds_copy.items():
                    ud.ter_filho()
                    ud.buscar_ud(media, varieties_list, True, False)
                    ud.incrementar_idade()
                    ud.death_probability(prob_morte)
            #Get the distribution, community diversity and mean by UD
            #print(len([ud for id, ud in uds_copy.items() if ud.activa]))
            #Get the distribution, community diversity and mean by UD
            total_varieties = []
            varieties_per_UD = []
            for ud in [ud for ud in UD_restricted_al_inh.uds.values() if ud.activa ==True]:
                varieties_per_UD.append(len(ud.varieties))
                for variety in ud.varieties:
                    total_varieties.append(variety.variety_id)
            #mean_ud = np.mean(varieties_per_UD)
            #diversity_comm_level = len(set(total_varieties))
            data_restricted_mean_ud.append(np.mean(varieties_per_UD))
            data_restricted_com_level.append(len(set(total_varieties)))
            data_restricted_total_varieties.append(total_varieties)
        data_inherit_restricted[inherit] = [data_restricted_mean_ud, data_restricted_com_level, data_restricted_total_varieties]
    
# Cuidado, no sobreescribir
    
# # Folder
# folder_name = 'results_inheritance_aleatory'
# if not os.path.exists(folder_name):
#     os.makedirs(folder_name)
# # Inherit data
# inherit_restricted_path = os.path.join(folder_name, 'inherit_data_restricted.json')
# # Save
# with open(inherit_restricted_path, 'w') as json_file:
#     json.dump(data_inherit_restricted, json_file, indent=4)
# # Confirmation message
# print("Data saved successfully to", inherit_restricted_path)